## SVM on RBM features

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 10, 7.5
plt.rcParams['axes.grid'] = True
plt.gray();

import sys
import numpy as np
sys.path.append('../scripts')
import datasets

#### Cluster definition and Initialization

In [2]:
from ipyparallel import Client
client = Client()
lb_view = client.load_balanced_view()

In [3]:
len(client)

145

#### Model definition

In [4]:
from sklearn.svm import SVC
model = SVC()
model_keyword='rbm_svm'
svc_params = {
    'C': np.logspace(-4, 2, 6),
    'gamma': np.logspace(-4, 2, 5),
    'kernel':['linear','rbf']
}

In [5]:
from sklearn.datasets import load_digits
from mmap_utils import persist_cv_splits
from sklearn.preprocessing import StandardScaler

data=datasets.load_rbm()
scaler = StandardScaler()
X_train = scaler.fit_transform(data.X_train)
X_test = scaler.transform(data.X_test)
Y_train= data.y_train
Y_test= data.y_test

digits_split_filenames = persist_cv_splits(X_train, Y_train,
    name=model_keyword, random_state=4)

In [6]:
import model_selection
search = model_selection.RandomizedGridSeach(lb_view)
search.launch_for_splits(model, svc_params, digits_split_filenames)

Progress: 00% (000/180)

In [13]:
print(search.report())

RemoteError: EngineError(Engine '112d36b3-e69a-4249-b07a-aef9631c919a' died while running task u'ed3ad5fa-b971-47f0-a5cb-cee79df55fa7')

Traceback (most recent call last):
  File "/usr/lib/python2.7/logging/__init__.py", line 872, in emit
    stream.write(fs % msg)
IOError: [Errno 32] Broken pipe
Logged from file ipkernel.py, line 207


In [ ]:
search.boxplot_parameters(display_train=False)

In [ ]:
def save_model(model,params,X_train,Y_train):
    model.set_params(**params)
    l=model.fit(X_train,Y_train)
    return l

In [ ]:
mod=save_model(model,search.find_bests(1)[0][-1],X_train,Y_train)

In [ ]:
classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
Y_pred=mod.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print 'Performance on Test set: %0.2f'% (accuracy_score(Y_test,Y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print 'Classification Statistics:'
print classification_report(Y_test,Y_pred,target_names=classes)

conf = confusion_matrix(Y_test, Y_pred)
plt.imshow(conf, cmap='binary',interpolation='nearest')
plt.title('Confusion Matrix'); plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45);plt.yticks(tick_marks, classes)
plt.tight_layout();plt.ylabel('True label');plt.xlabel('Predicted label')
font = {'color':  'blue','stretch':'extra-expanded','weight':0,'size': 10}
for i,j in [(n,m) for n in range(len(classes)) for m in range(len(classes))]:
    plt.text(j,i,str(conf[i,j]),font)
